In [29]:
#2020
#f3d86a6de7f8cf4262f3b272206e26a9275cd1d8
from architecture import *
import glob, math, time, datetime
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from egnn_pytorch import EGNN_Network
from torch.optim.lr_scheduler import ReduceLROnPlateau
import wandb
import os
os.environ["WANDB_MODE"] = "offline"
#optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


print("NPZ in",len(train_ds), len(val_ds), (time.time() - to) / 60, "min")
def collate_graphs(batch):
    return batch

train_loader = DataLoader(
    train_ds, batch_size=batch_size, shuffle=True,
    num_workers=0, pin_memory=True, collate_fn=collate_graphs
)
val_loader = DataLoader(
    val_ds, batch_size=batch_size, shuffle=False,
    num_workers=0, pin_memory=True, collate_fn=collate_graphs
)
print("pinned and reaady",len(train_loader), len(val_loader), (time.time() - to) / 60, "min")
# 2) model pieces

# --- EGNN + FFN + residual block ---
# --- EGNN + FFN + residual block ---




# 3) instantiate everything
dim, basis = 10, 64 #scale to 3,16 at least # dim must be divisible by 2
depth=2 #scale to 2, at least
hidden_dim=3
num_heads=dim + basis 
num_edge_tokens=256
num_global_tokens=256
dropout=0.02
cutoff=20.0
epochs=150
num_neighbors=2
net   = build_egnn(dim,depth,hidden_dim,num_neighbors,num_edge_tokens,num_global_tokens,dropout).to(device)
A     = PositionalEncoding(dim+basis).to(device)
mha   = AttentionBlock(embed_dim=dim+basis, num_heads=num_heads, hidden_dim=hidden_dim).to(device)
RBF   = LearnableRBF(num_basis=basis, cutoff=cutoff).to(device) 
model = TinyRegressor(in_channels=basis+dim,hidden_dim=dim+basis).to(device)
runid=datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

#8)

criterion = nn.L1Loss()
optimizer = torch.optim.AdamW([
    {"params": net.parameters(),   "lr":5e-3},
    {"params": mha.parameters(),   "lr":5e-3},
    {"params": model.parameters(), "lr":5e-3},
    {"params": RBF.parameters(),   "lr":5e-3}])
#scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.001, max_lr=0.1)
print("scheduler 5e-3. num nearest neiighbors 2 cutoff 9 num heads = in_dim.")#
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=.99, patience=0, cooldown=0, min_lr=1e-8, verbose=False)
# 4) training + validation
train_hist, val_hist = [], []
print("runid:",runid)
print("dim = num heads embeded dim:", dim)
print("depth",depth)
print("hidden_dim:",hidden_dim)
print("basis", basis)
print(f"hidden dim 8. lower optimizer and inc batch size to 25 from 5. changed mha to act on all and more thx to mam and chat anon and more")#rbf cut 4 same as real cutouts, only .01 dropout regularization, batch size 250. full val, {len(train_loader)} train")
print("")
#print(optimizer.state_dict())
print("FIX SCHEDULER Max schedular cool 0 wait 0 min 1e-8 patience zero cooldown zero")
print(criterion)
"""out1 N, dim
attn out N, N, 2
pairwise Ds 1,1,N
rbf 1 1 N n_rbf
rbf N basis
model input N 3 3
gnode N 1
gemb 1 nasis+1 n 
pooled 1 1 1
"""
# Start a new wandb run to track this script.
run = wandb.init(
    # Set the wandb entity where your project will be logged (generally your team name).
    entity="Biomodeling",
    # Set the wandb project where this run will be logged.
    project="Overfit PKEGNN on num_atoms = 500",
    # Track hyperparameters and run metadata.
    config={"runid": runid,
        "learning_rate": [op["lr"] for op in optimizer.param_groups], #net mha model rbf
        "dataset": f"inputs3 (num neighbors = 500) {len(paths) + len(tpaths)}",
        "batch size": batch_size,
        "epochs": epochs,
        "dim": dim,
        "depth": depth,
        "basis": basis,
        "num edge and global tokens": [num_edge_tokens,num_global_tokens],
        "dropout": [dropout, 0.03], #egnn p.enc. 
        "rbf cutoff": cutoff,
        "scheduler": [scheduler.min_lrs,scheduler.cooldown,scheduler.factor,"updates per epoch"],
        "loss": criterion,
        "architecture": b"EGNN (1 blocks) --> coords thru rbf --> concat rbf_out + egnn_out[0] ==> MHA --> model.linear --> pool > all protein's outputs thru model.linear2"})
vl,tl=[],[]
for epoch in range(epochs):
    print(f"Epoch {epoch}")
    net.train(); mha.train(); model.train(); RBF.train()
    epoch_train_losses = []

    # training
    for batch in train_loader:
        optimizer.zero_grad()
        for zs, xs, ys in batch:
            outs = []
            for z_t, x_t in zip(zs, xs):
                z_t = z_t.to(device).unsqueeze(0)
                x_t = x_t.to(device).unsqueeze(0)
                with autocast():
                    out1, coords = net(z_t, x_t)

                    #dmat = pairwise_distances(coords)
                    rbf  = RBF(pairwise_distances(coords))
                    b=torch.concat((rbf[:,0].T,out1[0].T.unsqueeze(2)))
                    en = A(b.permute(1,2,0)) #encoding
                    c=model.lin(mha(en)[0])

                    pooled = model.scalar_pool(c)[0][0] 

                outs.append(pooled)
            
            preds  = torch.stack(outs).to(device)
            #print(preds.shape,"preds")
            
            target = torch.hstack(ys).to(device).flatten()
            with autocast():
                #preds2=model.conv5(preds.unsqueeze(1)).to(device).flatten()
                pred3=model.lin2(preds).flatten().to(device)
            #pred3=model.relu(preds)
            #pred3=model.rel(preds)
            loss = criterion(pred3, target)
            #print(loss)
            #print(pred3)
            if use_amp:
                #with autocast():
                scaler.scale(loss).backward()
                #scheduler.step(loss)
                
            else:
                loss.backward()
            epoch_train_losses.append(loss.item())

        if use_amp:
            scaler.step(optimizer); scaler.update()
        else:
            optimizer.step()

    #train_hist.append(np.mean(epoch_train_losses))
    

    # validation
    print(loss)
    net.eval(); mha.eval(); model.eval(); RBF.eval()
    epoch_val_losses = []
    with torch.no_grad():
        for batch in val_loader:
            for zs, xs, ys in batch:
                outs = []
                for z_t, x_t in zip(zs, xs):
                    z_t = z_t.to(device).unsqueeze(0)
                    x_t = x_t.to(device).unsqueeze(0)
                    with autocast():
                        out1, coords = net(z_t, x_t)
                        rbf  = RBF(pairwise_distances(coords))
                        b=torch.concat((rbf[:,0].T,out1[0].T.unsqueeze(2)))
                        en = A(b.permute(1,2,0)) #encoding
                        c=model.lin(mha(en)[0])

                    pooled = model.scalar_pool(c)[0][0] 
                    outs.append(pooled)
                
                preds  = torch.stack(outs).to(device)
                #print(preds.shape,"preds")
                
                target = torch.hstack(ys).to(device).flatten()
                
                with autocast():
                    #preds2=model.conv5(preds.unsqueeze(1)).to(device).flatten()
                    pred3=model.lin2(preds).flatten().to(device)
                loss = criterion(pred3, target)
                epoch_val_losses.append(loss)
                #loss = torch.mean(epoch_val_losses)
                #scheduler.step(loss)  # val_loss = your average validation loss
                #scheduler.step(loss) 
            #e
            #print(val_loss.item())

                    # 5) save a single timestamped checkpoint
    
    loss=torch.mean(torch.tensor(epoch_val_losses))
    scheduler.step(loss) 
    tl.append(epoch_train_losses)
    vl.append(epoch_val_losses)
    elapsed_min = (time.time() - t0) / 60
    


    #print(torch.mean(torch.tensor(epoch_train_losses)),loss)
    
# 5) save a single timestamped checkpoint
    elapsed_min = (time.time() - t0) / 60
    print("pooled",pooled, pred3[-1],target[-1])
    print(elapsed_min,"min")
                #print(val_loss.item())

                    # 5) save a single timestamped checkpoint
    
    #loss=torch.mean(torch.tensor(epoch_val_losses))
    #scheduler.step(loss) 
    vloss,tloss=loss.item(),torch.mean(torch.tensor(epoch_train_losses)).item()
    #log
    run.log({"tloss": tloss, "vloss": vloss, "elapsed_time": elapsed_min})
    #elapsed_min = (time.time() - t0) / 60
    #print(len(epoch_train_losses),len(epoch_val_losses))
    #timestamp   = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    checkpoint = {
        "epoch":         epoch+1,
        "elapsed_min":   elapsed_min,
    #    "net":           net.state_dict(),
    #    "mha":           mha.state_dict(),
    #    "model":         model.state_dict(),
    #    "rbf":           RBF.state_dict(),
    #    "optimizer":     optimizer.state_dict(),
        "train_history": tl,
        "val_history":   vl}
    #}
    torch.save(checkpoint, f"./{runid}-checkpoint.pt")
    #print(f"Saved checkpoint_{timestamp}.pt ({elapsed_min:.1f} min)")

    #val_hist.append(loss.item())
    print(f" → avg train loss: {tloss:.4f}")
    print(f" → avg   val loss: {vloss:.4f}")
run.finish()

# 5) save a single timestamped checkpoint
elapsed_min = (time.time() - t0) / 60
timestamp   = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
checkpoint = {
    "epoch":         epoch+1,
    "elapsed_min":   elapsed_min,
    "net":           net.state_dict(),
    "mha":           mha.state_dict(),
    "model":         model.state_dict(),
    "rbf":           RBF.state_dict(),
    "optimizer":     optimizer.state_dict(),
    "scheduler":     scheduler.state_dict(),
    "train_history": tl,
    "val_history":   vl,
}
torch.save(checkpoint, f"./{runid}-checkpoint_{timestamp}.pt")
print(f"Saved checkpoint_{timestamp}.pt ({elapsed_min:.1f} min)")
#os.system("wandb sync --include-offline --sync-all wandb")





ModuleNotFoundError: No module named 'wandb'

In [1]:
from architecture import *

/home/jrhoernschemeyer/.local/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
for batch in val_loader:
    print("batch OK, shapes:", [t.shape for t in batch[:3]], batch[3].shape)
    break


batch OK, shapes: [torch.Size([5, 102, 20]), torch.Size([5, 102, 20, 3]), torch.Size([5, 102])] torch.Size([5, 102])


In [4]:
import datetime, time
from architecture import *
import torch
import glob, math, time, datetime
import numpy as np
import torch
import torch.nn as nn
from egnn_pytorch import EGNN
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from egnn_pytorch import EGNN_Network
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os
# 0) start timer
t0 = time.time()
N_NEIGHBORS = 15
BATCH_SIZE  =  1           # now safe to increase
PIN_MEMORY  = torch.cuda.is_available()
# reproducibility + device
torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# decide AMP only on GP0
use_amp = (device.type == "cuda")
if use_amp:
    scaler = GradScaler()
else:
    class DummyCM:
        def __enter__(self): pass
        def __exit__(self, *args): pass
    autocast = DummyCM
    scaler   = None

def init_model(dim,depth,hidden_dim,num_neighbors,num_edge_tokens,num_global_tokens,dropout):
    
    def build_egnn(dim,depth,hidden_dim,num_neighbors, num_edge_tokens,num_global_tokens,dropout):
        return StackedEGNN(
            dim=dim, depth=depth, hidden_dim=hidden_dim,
            dropout=dropout,
            num_positions=1000, num_tokens=118,
            num_nearest_neighbors=num_neighbors,
            norm_coors=True,
            num_edge_tokens=num_edge_tokens,
            num_global_tokens=num_global_tokens
        )
    net   = build_egnn(dim,depth,hidden_dim,num_neighbors,num_edge_tokens,num_global_tokens,dropout).to(device)
    mha   = AttentionBlock(embed_dim=dim+basis, num_heads=num_heads, hidden_dim=hidden_dim).to(device)
    RBF   = LearnableRBF(num_basis=basis, cutoff=cutoff).to(device) 
    return net, mha, RBF
#net,mha,RBF=init_model
# 3) instantiate everything
dim, basis = 2, 8 #scale to 3,16 at least # dim must be divisible by 2
depth=2 #scale to 2, at least
hidden_dim=3
num_heads=dim + basis 
num_edge_tokens=256
num_global_tokens=256
dropout=0.02
cutoff=10.0
num_neighbors=2


runid=datetime.datetime.now().strftime("%Y%m%d_%H%M%S")


from sklearn.neighbors import NearestNeighbors
from torch.utils.data import Dataset, DataLoader
import numpy as np, torch, glob

class InMemoryHoodDataset(Dataset):
    """
    Loads *.npz files, constructs fixed-size neighbourhoods around each
    site (anchor) and stores the result entirely in RAM.

    For a protein with S sites the shapes are
        z   : (S, N_NEIGHBORS)      int32
        pos : (S, N_NEIGHBORS, 3)   float32
        y   : (S,)                  float32
    """
    def __init__(self, paths, n_neighbors=N_NEIGHBORS, pin_memory=PIN_MEMORY):
        super().__init__()
        self.data = []
        nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm="brute")

        for p in paths:
            try:
                dat = np.load(p, allow_pickle=True)
                z_all   = dat["z"].astype(np.int32)        # (N,)
                pos_all = dat["pos"].astype(np.float32)    # (N,3)
                sites   = dat["sites"].astype(np.float32)  # (S,3)
                y       = dat["pks"].astype(np.float32)    # (S,)

                if len(sites) == 0:
                    continue  # skip empty entries

                nbrs.fit(pos_all)
                idx = nbrs.kneighbors(sites, return_distance=False)   # (S, N_NEIGHBORS)

                z_hood   = torch.from_numpy(z_all[idx])            # (S,N_NEIGHBORS)
                pos_hood = torch.from_numpy(pos_all[idx])          # (S,N_NEIGHBORS,3)
                y        = torch.from_numpy(y)                     # (S,)

                if pin_memory:
                    z_hood   = z_hood.pin_memory()
                    pos_hood = pos_hood.pin_memory()
                    y        = y.pin_memory()

                self.data.append((z_hood, pos_hood, y))
            except Exception as e:
                print(f"skipping {p}: {e}")

    def __len__(self):             return len(self.data)
    def __getitem__(self, idx):    return self.data[idx]

# ---------------------------------------------------------------------
# 2) collate function  -------------------------------------------------
# ---------------------------------------------------------------------
def pad_collate(batch):
    """
    Pads the variable-length site dimension so the batch can be stacked
    into one tensor.  A boolean mask keeps track of which elements are
    real data (True) vs. padding (False).
    """
    # batch = list[(z,pos,y), ...]         len = B
    B               = len(batch)
    S_max           = max(item[0].shape[0] for item in batch)   # longest protein
    device          = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    zs   = torch.zeros (B, S_max, N_NEIGHBORS ,   dtype=torch.int32 , device=device)
    pos  = torch.zeros (B, S_max, N_NEIGHBORS ,3, dtype=torch.float32, device=device)
    ys   = torch.full  ((B, S_max),  float("nan"), dtype=torch.float32, device=device)
    #ys   = torch.full  (B, S_max,               float("nan"),        dtype=torch.float32, device=device)
    mask = torch.zeros (B, S_max,                                   dtype=torch.bool,     device=device)

    for b,(z,pos_b,y) in enumerate(batch):
        S = z.shape[0]
        zs  [b, :S] = z.to(device)
        pos [b, :S] = pos_b.to(device)
        ys  [b, :S] = y.to(device)
        mask[b, :S] = True

    return zs, pos, ys, mask             # shapes – see above

# ---------------------------------------------------------------------
# 3) data loaders ------------------------------------------------------
# ---------------------------------------------------------------------

# 0) parameters you might want to expose at the top of the script
# ---------------------------------------------------------------------

# 3) data loaders ------------------------------------------------------
# ---------------------------------------------------------------------
np.random.seed(0)
all_paths = glob.glob("../../../data/pkegnn_INS/inputs/*.npz")
np.random.shuffle(all_paths)
train_paths, val_paths = all_paths[:20], all_paths[20:30]

train_ds = InMemoryHoodDataset(train_paths)
val_ds   = InMemoryHoodDataset(val_paths)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE,
                          shuffle=True,  collate_fn=pad_collate,
                          num_workers=0, pin_memory=PIN_MEMORY)
val_loader   = DataLoader(val_ds  , batch_size=BATCH_SIZE,
                          shuffle=False, collate_fn=pad_collate,
                          num_workers=0, pin_memory=PIN_MEMORY)



# ---------------------------------------------------------------------
# 4) model pieces ------------------------------------------------------
# ---------------------------------------------------------------------
egnn_net = StackedEGNN(dim=dim, depth=depth, hidden_dim=hidden_dim,
                       dropout=dropout, num_positions=1000, num_tokens=118,
                       num_nearest_neighbors=num_neighbors,
                       norm_coors=True,
                       num_edge_tokens=num_edge_tokens,
                       num_global_tokens=num_global_tokens).to(device)

rbf_layer = LearnableRBF(num_basis=basis, cutoff=cutoff).to(device)
mha_layer = AttentionBlock(embed_dim=dim + basis,
                           num_heads=num_heads,
                           hidden_dim=hidden_dim).to(device)
pred_head = nn.Linear(dim + basis, 1).to(device)

protein_egnn=EGNN(dim=1,update_coors=False,num_nearest_neighbors=8)
criterion = nn.L1Loss()
optimizer = torch.optim.AdamW(
    list(egnn_net.parameters()) +
    list(rbf_layer.parameters()) +
    list(mha_layer.parameters()) +
    list(pred_head.parameters()) +
    list(protein_egnn.parameters()),
    lr=5e-3
)

epochs = 1  # or whatever you like

# ---------------------------------------------------------------------
# 5) forward for a *compressed* batch (R residues, N neighbours)
# ---------------------------------------------------------------------
# ---------------------------------------------------------------------
# 5) single-path forward – no shape guessing, no branching
# ---------------------------------------------------------------------
def forward_residues(z_r, x_r):
    """
    z_r : (R, N)       int32   – atomic numbers for R residues
    x_r : (R, N, 3)    float32 – coordinates
    returns (R, dim + basis)   – per-residue embeddings
    """
    # ---------- EGNN ----------
    h_out, coords = egnn_net(z_r, x_r)          # h_out is [tensor] or tensor
    h = h_out[0] if isinstance(h_out, (list, tuple)) else h_out   # (R, N, dim)

    # ---------- RBF on *input* coords (already (R,N,3)) ----------
    #d   = torch.cdist(x_r, x_r)            # (R, N, N)
    
    rbf = rbf_layer(coords)                     # (R, N, N, basis)

    # ---------- concat & attention ----------
    h0  = h.transpose(1, 2)                # (R, dim,   N)
    r0  = rbf[:, 0].transpose(1, 2)        # (R, basis, N)
    tok = torch.cat((r0, h0), dim=1)       # (R, dim+basis, N)

    tok, _ = mha_layer(tok.permute(2, 0, 1))   # (N, R, C) → attn(+PE)
    tok    = tok.permute(1, 0, 2).max(dim=1).values   # (R, C) max over neighbours
    return tok,    coords,  coords.mean(dim=1).unsqueeze(0)                               # (R, dim + basis)
                                         # (R, dim+basis)

# ---------------------------------------------------------------------
# 6) training / validation loop ---------------------------------------
# ---------------------------------------------------------------------
for epoch in range(epochs):
    # ======== TRAIN ========
    egnn_net.train(); rbf_layer.train(); mha_layer.train(); pred_head.train()
    tr_losses = []

    for z, x, y, mask in train_loader:                 # z:(B,S,N)  mask:(B,S)
        # compress away padding →  (R, N), (R, N, 3), (R,)
        valid      = mask.view(-1)
        z_res      = z.view(-1, z.size(2))[valid].to(device)
        x_res      = x.view(-1, x.size(2), 3)[valid].to(device)
        y_res      = y.view(-1)[valid].to(device)

        optimizer.zero_grad()

        #model
        feats, centroids, coords = forward_residues(z_res, x_res)         # (R, C)
        
        preds = pred_head(feats)       
        t=preds.unsqueeze(0)
        #preds=protein_egnn(t,centroids)[0]

        loss  = criterion(preds.flatten(), y_res)

        if use_amp:
            scaler.scale(loss).backward()
            scaler.step(optimizer); scaler.update()
        else:
            loss.backward(); optimizer.step()

        tr_losses.append(loss.item())

    print(f"Epoch {epoch:3d} | train L1 = {np.mean(tr_losses):.4f}")

    
    # ======== VALID ========
    egnn_net.eval(); rbf_layer.eval(); mha_layer.eval(); pred_head.eval()
    vl_losses = []

    with torch.no_grad():
        for z, x, y, mask in val_loader:
            valid   = mask.view(-1)
            z_res   = z.view(-1, z.size(2))[valid].to(device)
            x_res   = x.view(-1, x.size(2), 3)[valid].to(device)
            y_res   = y.view(-1)[valid].to(device)

                #model
            feats, centroids, coords = forward_residues(z_res, x_res)         # (R, C)
            
            preds = pred_head(feats)       
            t=preds.unsqueeze(0)
            #preds=protein_egnn(t,centroids)[0]

            loss  = criterion(preds.flatten(), y_res)
            vl_losses.append(loss.item())

    print(f"              |  val L1 = {np.mean(vl_losses):.4f}")


TypeError: forward() missing 1 required positional argument: 'coords'

In [3]:
import datetime, time
from architecture import *
import torch
import glob, math, time, datetime
import numpy as np
import torch
import torch.nn as nn
from egnn_pytorch import EGNN
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from egnn_pytorch import EGNN_Network
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os
# 0) start timer
t0 = time.time()
N_NEIGHBORS = 15
BATCH_SIZE  =  1           # now safe to increase
PIN_MEMORY  = torch.cuda.is_available()
# reproducibility + device
torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# decide AMP only on GP0
use_amp = (device.type == "cuda")
if use_amp:
    scaler = GradScaler()
else:
    class DummyCM:
        def __enter__(self): pass
        def __exit__(self, *args): pass
    autocast = DummyCM
    scaler   = None

def init_model(dim,depth,hidden_dim,num_neighbors,num_edge_tokens,num_global_tokens,dropout):
    
    def build_egnn(dim,depth,hidden_dim,num_neighbors, num_edge_tokens,num_global_tokens,dropout):
        return StackedEGNN(
            dim=dim, depth=depth, hidden_dim=hidden_dim,
            dropout=dropout,
            num_positions=1000, num_tokens=118,
            num_nearest_neighbors=num_neighbors,
            norm_coors=True,
            num_edge_tokens=num_edge_tokens,
            num_global_tokens=num_global_tokens
        )
    net   = build_egnn(dim,depth,hidden_dim,num_neighbors,num_edge_tokens,num_global_tokens,dropout).to(device)
    mha   = AttentionBlock(embed_dim=dim+basis, num_heads=num_heads, hidden_dim=hidden_dim).to(device)
    RBF   = LearnableRBF(num_basis=basis, cutoff=cutoff).to(device) 
    return net, mha, RBF
#net,mha,RBF=init_model
# 3) instantiate everything
dim, basis = 2, 8 #scale to 3,16 at least # dim must be divisible by 2
depth=2 #scale to 2, at least
hidden_dim=3
num_heads=dim + basis 
num_edge_tokens=256
num_global_tokens=256
dropout=0.02
cutoff=10.0
num_neighbors=2


runid=datetime.datetime.now().strftime("%Y%m%d_%H%M%S")


from sklearn.neighbors import NearestNeighbors
from torch.utils.data import Dataset, DataLoader
import numpy as np, torch, glob

class InMemoryHoodDataset(Dataset):
    """
    Loads *.npz files, constructs fixed-size neighbourhoods around each
    site (anchor) and stores the result entirely in RAM.

    For a protein with S sites the shapes are
        z   : (S, N_NEIGHBORS)      int32
        pos : (S, N_NEIGHBORS, 3)   float32
        y   : (S,)                  float32
    """
    def __init__(self, paths, n_neighbors=N_NEIGHBORS, pin_memory=PIN_MEMORY):
        super().__init__()
        self.data = []
        nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm="brute")

        for p in paths:
            try:
                dat = np.load(p, allow_pickle=True)
                z_all   = dat["z"].astype(np.int32)        # (N,)
                pos_all = dat["pos"].astype(np.float32)    # (N,3)
                sites   = dat["sites"].astype(np.float32)  # (S,3)
                y       = dat["pks"].astype(np.float32)    # (S,)

                if len(sites) == 0:
                    continue  # skip empty entries

                nbrs.fit(pos_all)
                idx = nbrs.kneighbors(sites, return_distance=False)   # (S, N_NEIGHBORS)

                z_hood   = torch.from_numpy(z_all[idx])            # (S,N_NEIGHBORS)
                pos_hood = torch.from_numpy(pos_all[idx])          # (S,N_NEIGHBORS,3)
                y        = torch.from_numpy(y)                     # (S,)

                if pin_memory:
                    z_hood   = z_hood.pin_memory()
                    pos_hood = pos_hood.pin_memory()
                    y        = y.pin_memory()

                self.data.append((z_hood, pos_hood, y))
            except Exception as e:
                print(f"skipping {p}: {e}")

    def __len__(self):             return len(self.data)
    def __getitem__(self, idx):    return self.data[idx]

# ---------------------------------------------------------------------
# 2) collate function  -------------------------------------------------
# ---------------------------------------------------------------------
def pad_collate(batch):
    """
    Pads the variable-length site dimension so the batch can be stacked
    into one tensor.  A boolean mask keeps track of which elements are
    real data (True) vs. padding (False).
    """
    # batch = list[(z,pos,y), ...]         len = B
    B               = len(batch)
    S_max           = max(item[0].shape[0] for item in batch)   # longest protein
    device          = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    zs   = torch.zeros (B, S_max, N_NEIGHBORS ,   dtype=torch.int32 , device=device)
    pos  = torch.zeros (B, S_max, N_NEIGHBORS ,3, dtype=torch.float32, device=device)
    ys   = torch.full  ((B, S_max),  float("nan"), dtype=torch.float32, device=device)
    #ys   = torch.full  (B, S_max,               float("nan"),        dtype=torch.float32, device=device)
    mask = torch.zeros (B, S_max,                                   dtype=torch.bool,     device=device)

    for b,(z,pos_b,y) in enumerate(batch):
        S = z.shape[0]
        zs  [b, :S] = z.to(device)
        pos [b, :S] = pos_b.to(device)
        ys  [b, :S] = y.to(device)
        mask[b, :S] = True

    return zs, pos, ys, mask             # shapes – see above

# ---------------------------------------------------------------------
# 3) data loaders ------------------------------------------------------
# ---------------------------------------------------------------------

# 0) parameters you might want to expose at the top of the script
# ---------------------------------------------------------------------

# 3) data loaders ------------------------------------------------------
# ---------------------------------------------------------------------
np.random.seed(0)
all_paths = glob.glob("../../../data/pkegnn_INS/inputs/*.npz")
np.random.shuffle(all_paths)
train_paths, val_paths = all_paths[:20], all_paths[20:30]

train_ds = InMemoryHoodDataset(train_paths)
val_ds   = InMemoryHoodDataset(val_paths)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE,
                          shuffle=True,  collate_fn=pad_collate,
                          num_workers=0, pin_memory=PIN_MEMORY)
val_loader   = DataLoader(val_ds  , batch_size=BATCH_SIZE,
                          shuffle=False, collate_fn=pad_collate,
                          num_workers=0, pin_memory=PIN_MEMORY)



# ---------------------------------------------------------------------
# 4) model pieces ------------------------------------------------------
# ---------------------------------------------------------------------
egnn_net = StackedEGNN(dim=dim, depth=depth, hidden_dim=hidden_dim,
                       dropout=dropout, num_positions=1000, num_tokens=118,
                       num_nearest_neighbors=num_neighbors,
                       norm_coors=True,
                       num_edge_tokens=num_edge_tokens,
                       num_global_tokens=num_global_tokens).to(device)

rbf_layer = LearnableRBF(num_basis=basis, cutoff=cutoff).to(device)
mha_layer = AttentionBlock(embed_dim=dim + basis,
                           num_heads=num_heads,
                           hidden_dim=hidden_dim).to(device)
pred_head = nn.Linear(dim + basis, 1).to(device)

protein_egnn=EGNN(dim=1,update_coors=False,num_nearest_neighbors=8)
criterion = nn.L1Loss()
optimizer = torch.optim.AdamW(
    list(egnn_net.parameters()) +
    list(rbf_layer.parameters()) +
    list(mha_layer.parameters()) +
    list(pred_head.parameters()) +
    list(protein_egnn.parameters()),
    lr=5e-3
)

epochs = 1  # or whatever you like

# ---------------------------------------------------------------------
# 5) forward for a *compressed* batch (R residues, N neighbours)
# ---------------------------------------------------------------------
# ---------------------------------------------------------------------
# 5) single-path forward – no shape guessing, no branching
# ---------------------------------------------------------------------
def forward_residues(z_r, x_r):
    """
    z_r : (R, N)       int32   – atomic numbers for R residues
    x_r : (R, N, 3)    float32 – coordinates
    returns (R, dim + basis)   – per-residue embeddings
    """
    # ---------- EGNN ----------
    h_out, coords = egnn_net(z_r, x_r)          # h_out is [tensor] or tensor
    h = h_out[0] if isinstance(h_out, (list, tuple)) else h_out   # (R, N, dim)

    # ---------- RBF on *input* coords (already (R,N,3)) ----------
    #d   = torch.cdist(x_r, x_r)            # (R, N, N)
    centroids=coords.mean(dim=0)
    rbf = rbf_layer(centroids,coords)                     # (R, N, N, basis)

    # ---------- concat & attention ----------
    h0  = h.transpose(1, 2)                # (R, dim,   N)
    r0  = rbf.transpose(1, 2)        # (R, basis, N)
    tok = torch.cat((r0, h0), dim=1)       # (R, dim+basis, N)

    tok, _ = mha_layer(tok.permute(2, 0, 1))   # (N, R, C) → attn(+PE)
    tok    = tok.permute(1, 0, 2).max(dim=1).values   # (R, C) max over neighbours
    return tok,    centroids                              # (R, dim + basis)
                                         # (R, dim+basis)

# ---------------------------------------------------------------------
# 6) training / validation loop ---------------------------------------
# ---------------------------------------------------------------------
for epoch in range(epochs):
    # ======== TRAIN ========
    egnn_net.train(); rbf_layer.train(); mha_layer.train(); pred_head.train()
    tr_losses = []

    for z, x, y, mask in train_loader:                 # z:(B,S,N)  mask:(B,S)
        # compress away padding →  (R, N), (R, N, 3), (R,)
        valid      = mask.view(-1)
        z_res      = z.view(-1, z.size(2))[valid].to(device)
        x_res      = x.view(-1, x.size(2), 3)[valid].to(device)
        y_res      = y.view(-1)[valid].to(device)

        optimizer.zero_grad()

        #model
        feats, centroids = forward_residues(z_res, x_res)         # (R, C)
        
        preds = pred_head(feats)       
        t=preds.unsqueeze(0)
        #preds=protein_egnn(t,centroids)[0]

        loss  = criterion(preds.flatten(), y_res)

        if use_amp:
            scaler.scale(loss).backward()
            scaler.step(optimizer); scaler.update()
        else:
            loss.backward(); optimizer.step()

        tr_losses.append(loss.item())

    print(f"Epoch {epoch:3d} | train L1 = {np.mean(tr_losses):.4f}")

    
    # ======== VALID ========
    egnn_net.eval(); rbf_layer.eval(); mha_layer.eval(); pred_head.eval()
    vl_losses = []

    with torch.no_grad():
        for z, x, y, mask in val_loader:
            valid   = mask.view(-1)
            z_res   = z.view(-1, z.size(2))[valid].to(device)
            x_res   = x.view(-1, x.size(2), 3)[valid].to(device)
            y_res   = y.view(-1)[valid].to(device)

                #model
            feats, centroids = forward_residues(z_res, x_res)         # (R, C)
            
            preds = pred_head(feats)       
            t=preds.unsqueeze(0)
            #preds=protein_egnn(t,centroids)[0]

            loss  = criterion(preds.flatten(), y_res)
            vl_losses.append(loss.item())

    print(f"              |  val L1 = {np.mean(vl_losses):.4f}")


Epoch   0 | train L1 = 1.2329
              |  val L1 = 1.2776


In [27]:
centroids.shape

torch.Size([121, 15, 3])

In [23]:
centroids.shape

torch.Size([121, 15, 3])

In [28]:
coords.mean(dim=0).shape

torch.Size([121, 3])

In [31]:
(coords.mean(dim=1) - coords).shape

torch.Size([1, 121, 3])

In [32]:
centroids=coords.mean(dim=1)
dist = torch.norm(centroids - coords, dim=-1)

In [24]:
coords.shape

torch.Size([1, 121, 3])

In [21]:
coords[0].mean(dim=1).shape

torch.Size([121])

In [104]:
import datetime, time
from architecture import *
import torch
import glob, math, time, datetime
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from egnn_pytorch import EGNN_Network
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os
# 0) start timer
t0 = time.time()
N_NEIGHBORS = 15
BATCH_SIZE  =  1           # now safe to increase
PIN_MEMORY  = torch.cuda.is_available()
# reproducibility + device
torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# decide AMP only on GP0
use_amp = (device.type == "cuda")
if use_amp:
    scaler = GradScaler()
else:
    class DummyCM:
        def __enter__(self): pass
        def __exit__(self, *args): pass
    autocast = DummyCM
    scaler   = None

def init_model(dim,depth,hidden_dim,num_neighbors,num_edge_tokens,num_global_tokens,dropout):
    
    def build_egnn(dim,depth,hidden_dim,num_neighbors, num_edge_tokens,num_global_tokens,dropout):
        return StackedEGNN(
            dim=dim, depth=depth, hidden_dim=hidden_dim,
            dropout=dropout,
            num_positions=1000, num_tokens=118,
            num_nearest_neighbors=num_neighbors,
            norm_coors=True,
            num_edge_tokens=num_edge_tokens,
            num_global_tokens=num_global_tokens
        )
    net   = build_egnn(dim,depth,hidden_dim,num_neighbors,num_edge_tokens,num_global_tokens,dropout).to(device)
    mha   = AttentionBlock(embed_dim=dim+basis, num_heads=num_heads, hidden_dim=hidden_dim).to(device)
    RBF   = LearnableRBF(num_basis=basis, cutoff=cutoff).to(device) 
    return net, mha, RBF
#net,mha,RBF=init_model
# 3) instantiate everything
dim, basis = 2, 8 #scale to 3,16 at least # dim must be divisible by 2
depth=2 #scale to 2, at least
hidden_dim=3
num_heads=dim + basis 
num_edge_tokens=256
num_global_tokens=256
dropout=0.02
cutoff=10.0
num_neighbors=2


runid=datetime.datetime.now().strftime("%Y%m%d_%H%M%S")


from sklearn.neighbors import NearestNeighbors
from torch.utils.data import Dataset, DataLoader
import numpy as np, torch, glob

class InMemoryHoodDataset(Dataset):
    """
    Loads *.npz files, constructs fixed-size neighbourhoods around each
    site (anchor) and stores the result entirely in RAM.

    For a protein with S sites the shapes are
        z   : (S, N_NEIGHBORS)      int32
        pos : (S, N_NEIGHBORS, 3)   float32
        y   : (S,)                  float32
    """
    def __init__(self, paths, n_neighbors=N_NEIGHBORS, pin_memory=PIN_MEMORY):
        super().__init__()
        self.data = []
        nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm="brute")

        for p in paths:
            try:
                dat = np.load(p, allow_pickle=True)
                z_all   = dat["z"].astype(np.int32)        # (N,)
                pos_all = dat["pos"].astype(np.float32)    # (N,3)
                sites   = dat["sites"].astype(np.float32)  # (S,3)
                y       = dat["pks"].astype(np.float32)    # (S,)

                if len(sites) == 0:
                    continue  # skip empty entries

                nbrs.fit(pos_all)
                idx = nbrs.kneighbors(sites, return_distance=False)   # (S, N_NEIGHBORS)

                z_hood   = torch.from_numpy(z_all[idx])            # (S,N_NEIGHBORS)
                pos_hood = torch.from_numpy(pos_all[idx])          # (S,N_NEIGHBORS,3)
                y        = torch.from_numpy(y)                     # (S,)

                if pin_memory:
                    z_hood   = z_hood.pin_memory()
                    pos_hood = pos_hood.pin_memory()
                    y        = y.pin_memory()

                self.data.append((z_hood, pos_hood, y))
            except Exception as e:
                print(f"skipping {p}: {e}")

    def __len__(self):             return len(self.data)
    def __getitem__(self, idx):    return self.data[idx]

# ---------------------------------------------------------------------
# 2) collate function  -------------------------------------------------
# ---------------------------------------------------------------------
def pad_collate(batch):
    """
    Pads the variable-length site dimension so the batch can be stacked
    into one tensor.  A boolean mask keeps track of which elements are
    real data (True) vs. padding (False).
    """
    # batch = list[(z,pos,y), ...]         len = B
    B               = len(batch)
    S_max           = max(item[0].shape[0] for item in batch)   # longest protein
    device          = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    zs   = torch.zeros (B, S_max, N_NEIGHBORS ,   dtype=torch.int32 , device=device)
    pos  = torch.zeros (B, S_max, N_NEIGHBORS ,3, dtype=torch.float32, device=device)
    ys   = torch.full  ((B, S_max),  float("nan"), dtype=torch.float32, device=device)
    #ys   = torch.full  (B, S_max,               float("nan"),        dtype=torch.float32, device=device)
    mask = torch.zeros (B, S_max,                                   dtype=torch.bool,     device=device)

    for b,(z,pos_b,y) in enumerate(batch):
        S = z.shape[0]
        zs  [b, :S] = z.to(device)
        pos [b, :S] = pos_b.to(device)
        ys  [b, :S] = y.to(device)
        mask[b, :S] = True

    return zs, pos, ys, mask             # shapes – see above

# ---------------------------------------------------------------------
# 3) data loaders ------------------------------------------------------
# ---------------------------------------------------------------------

# 0) parameters you might want to expose at the top of the script
# ---------------------------------------------------------------------

# 3) data loaders ------------------------------------------------------
# ---------------------------------------------------------------------
np.random.seed(0)
all_paths = glob.glob("../inputs/*.npz")
np.random.shuffle(all_paths)
train_paths, val_paths = all_paths[:20], all_paths[20:30]

train_ds = InMemoryHoodDataset(train_paths)
val_ds   = InMemoryHoodDataset(val_paths)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE,
                          shuffle=True,  collate_fn=pad_collate,
                          num_workers=0, pin_memory=PIN_MEMORY)
val_loader   = DataLoader(val_ds  , batch_size=BATCH_SIZE,
                          shuffle=False, collate_fn=pad_collate,
                          num_workers=0, pin_memory=PIN_MEMORY)



# ---------------------------------------------------------------------
# 4) model pieces ------------------------------------------------------
# ---------------------------------------------------------------------
egnn_net = StackedEGNN(dim=dim, depth=depth, hidden_dim=hidden_dim,
                       dropout=dropout, num_positions=1000, num_tokens=118,
                       num_nearest_neighbors=num_neighbors,
                       norm_coors=True,
                       num_edge_tokens=num_edge_tokens,
                       num_global_tokens=num_global_tokens).to(device)

rbf_layer = LearnableRBF(num_basis=basis, cutoff=cutoff).to(device)
mha_layer = AttentionBlock(embed_dim=dim + basis,
                           num_heads=num_heads,
                           hidden_dim=hidden_dim).to(device)
pred_head = nn.Linear(dim + basis, 1).to(device)

protein_egnn=EGNN(dim=1,update_coors=False,num_nearest_neighbors=8)
criterion = nn.L1Loss()
optimizer = torch.optim.AdamW(
    list(egnn_net.parameters()) +
    list(rbf_layer.parameters()) +
    list(mha_layer.parameters()) +
    list(pred_head.parameters()) +
    list(protein_egnn.parameters()),
    lr=5e-3
)

epochs = 1  # or whatever you like

# ---------------------------------------------------------------------
# 5) forward for a *compressed* batch (R residues, N neighbours)
# ---------------------------------------------------------------------
# ---------------------------------------------------------------------
# 5) single-path forward – no shape guessing, no branching
# ---------------------------------------------------------------------
def forward_residues(z_r, x_r):
    """
    z_r : (R, N)       int32   – atomic numbers for R residues
    x_r : (R, N, 3)    float32 – coordinates
    returns (R, dim + basis)   – per-residue embeddings
    """
    # ---------- EGNN ----------
    h_out, coords = egnn_net(z_r, x_r)          # h_out is [tensor] or tensor
    h = h_out[0] if isinstance(h_out, (list, tuple)) else h_out   # (R, N, dim)

    # ---------- RBF on *input* coords (already (R,N,3)) ----------
    #d   = torch.cdist(x_r, x_r)            # (R, N, N)
    rbf = rbf_layer(coords)                     # (R, N, N, basis)

    # ---------- concat & attention ----------
    h0  = h.transpose(1, 2)                # (R, dim,   N)
    r0  = rbf[:, 0].transpose(1, 2)        # (R, basis, N)
    tok = torch.cat((r0, h0), dim=1)       # (R, dim+basis, N)

    tok, _ = mha_layer(tok.permute(2, 0, 1))   # (N, R, C) → attn(+PE)
    tok    = tok.permute(1, 0, 2).max(dim=1).values   # (R, C) max over neighbours
    return tok,     coords.mean(dim=1).unsqueeze(0)                               # (R, dim + basis)
                                         # (R, dim+basis)

# ---------------------------------------------------------------------
# 6) training / validation loop ---------------------------------------
# ---------------------------------------------------------------------
for epoch in range(epochs):
    # ======== TRAIN ========
    egnn_net.train(); rbf_layer.train(); mha_layer.train(); pred_head.train()
    tr_losses = []

    for z, x, y, mask in train_loader:                 # z:(B,S,N)  mask:(B,S)
        # compress away padding →  (R, N), (R, N, 3), (R,)
        valid      = mask.view(-1)
        z_res      = z.view(-1, z.size(2))[valid].to(device)
        x_res      = x.view(-1, x.size(2), 3)[valid].to(device)
        y_res      = y.view(-1)[valid].to(device)

        optimizer.zero_grad()

        #model
        feats, centroids = forward_residues(z_res, x_res)         # (R, C)
        
        preds = pred_head(feats)       
        t=preds.unsqueeze(0)
        preds=protein_egnn(t,centroids)[0]

        loss  = criterion(preds.flatten(), y_res)

        if use_amp:
            scaler.scale(loss).backward()
            scaler.step(optimizer); scaler.update()
        else:
            loss.backward(); optimizer.step()

        tr_losses.append(loss.item())

    print(f"Epoch {epoch:3d} | train L1 = {np.mean(tr_losses):.4f}")

    
    # ======== VALID ========
    egnn_net.eval(); rbf_layer.eval(); mha_layer.eval(); pred_head.eval()
    vl_losses = []

    with torch.no_grad():
        for z, x, y, mask in val_loader:
            valid   = mask.view(-1)
            z_res   = z.view(-1, z.size(2))[valid].to(device)
            x_res   = x.view(-1, x.size(2), 3)[valid].to(device)
            y_res   = y.view(-1)[valid].to(device)

                #model
            feats, centroids = forward_residues(z_res, x_res)         # (R, C)
            
            preds = pred_head(feats)       
            t=preds.unsqueeze(0)
            preds=protein_egnn(t,centroids)[0]

            loss  = criterion(preds.flatten(), y_res)
            vl_losses.append(loss.item())

    print(f"              |  val L1 = {np.mean(vl_losses):.4f}")


Epoch   0 | train L1 = 1.2356
              |  val L1 = 1.1295


In [105]:
for epoch in range(20):
    # ======== TRAIN ========
    egnn_net.train(); rbf_layer.train(); mha_layer.train(); pred_head.train()
    tr_losses = []

    for z, x, y, mask in train_loader:                 # z:(B,S,N)  mask:(B,S)
        # compress away padding →  (R, N), (R, N, 3), (R,)
        valid      = mask.view(-1)
        z_res      = z.view(-1, z.size(2))[valid].to(device)
        x_res      = x.view(-1, x.size(2), 3)[valid].to(device)
        y_res      = y.view(-1)[valid].to(device)

        optimizer.zero_grad()

        #model
        feats, centroids = forward_residues(z_res, x_res)         # (R, C)
        
        preds = pred_head(feats)       
        t=preds.unsqueeze(0)
        preds=protein_egnn(t,centroids)[0]

        loss  = criterion(preds.flatten(), y_res)

        if use_amp:
            scaler.scale(loss).backward()
            scaler.step(optimizer); scaler.update()
        else:
            loss.backward(); optimizer.step()

        tr_losses.append(loss.item())

    print(f"Epoch {epoch:3d} | train L1 = {np.mean(tr_losses):.4f}")

    
    # ======== VALID ========
    egnn_net.eval(); rbf_layer.eval(); mha_layer.eval(); pred_head.eval()
    vl_losses = []

    with torch.no_grad():
        for z, x, y, mask in val_loader:
            valid   = mask.view(-1)
            z_res   = z.view(-1, z.size(2))[valid].to(device)
            x_res   = x.view(-1, x.size(2), 3)[valid].to(device)
            y_res   = y.view(-1)[valid].to(device)

                #model
            feats, centroids = forward_residues(z_res, x_res)         # (R, C)
            
            preds = pred_head(feats)       
            t=preds.unsqueeze(0)
            preds=protein_egnn(t,centroids)[0]

            loss  = criterion(preds.flatten(), y_res)
            vl_losses.append(loss.item())

    print(f"              |  val L1 = {np.mean(vl_losses):.4f}")


Epoch   0 | train L1 = 1.1770
              |  val L1 = 1.0956
Epoch   1 | train L1 = 1.1575
              |  val L1 = 1.0465
Epoch   2 | train L1 = 1.1255
              |  val L1 = 1.0123
Epoch   3 | train L1 = 1.1085
              |  val L1 = 1.0101
Epoch   4 | train L1 = 1.1098
              |  val L1 = 0.9833
Epoch   5 | train L1 = 1.1022
              |  val L1 = 0.9626
Epoch   6 | train L1 = 1.0896
              |  val L1 = 0.9661
Epoch   7 | train L1 = 1.0964
              |  val L1 = 0.9841
Epoch   8 | train L1 = 1.0933
              |  val L1 = 1.0415
Epoch   9 | train L1 = 1.1144
              |  val L1 = 1.0438
Epoch  10 | train L1 = 1.1100
              |  val L1 = 0.9604
Epoch  11 | train L1 = 1.0953
              |  val L1 = 0.9550
Epoch  12 | train L1 = 1.0857
              |  val L1 = 0.9498
Epoch  13 | train L1 = 1.0760
              |  val L1 = 0.9426
Epoch  14 | train L1 = 1.0604
              |  val L1 = 0.9507
Epoch  15 | train L1 = 1.0770
              |  val L1 =

without hierachel scalar head

In [106]:
import datetime, time
from architecture import *
import torch
import glob, math, time, datetime
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from egnn_pytorch import EGNN_Network
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os
# 0) start timer
t0 = time.time()
N_NEIGHBORS = 15
BATCH_SIZE  =  1           # now safe to increase
PIN_MEMORY  = torch.cuda.is_available()
# reproducibility + device
torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# decide AMP only on GP0
use_amp = (device.type == "cuda")
if use_amp:
    scaler = GradScaler()
else:
    class DummyCM:
        def __enter__(self): pass
        def __exit__(self, *args): pass
    autocast = DummyCM
    scaler   = None

def init_model(dim,depth,hidden_dim,num_neighbors,num_edge_tokens,num_global_tokens,dropout):
    
    def build_egnn(dim,depth,hidden_dim,num_neighbors, num_edge_tokens,num_global_tokens,dropout):
        return StackedEGNN(
            dim=dim, depth=depth, hidden_dim=hidden_dim,
            dropout=dropout,
            num_positions=1000, num_tokens=118,
            num_nearest_neighbors=num_neighbors,
            norm_coors=True,
            num_edge_tokens=num_edge_tokens,
            num_global_tokens=num_global_tokens
        )
    net   = build_egnn(dim,depth,hidden_dim,num_neighbors,num_edge_tokens,num_global_tokens,dropout).to(device)
    mha   = AttentionBlock(embed_dim=dim+basis, num_heads=num_heads, hidden_dim=hidden_dim).to(device)
    RBF   = LearnableRBF(num_basis=basis, cutoff=cutoff).to(device) 
    return net, mha, RBF
#net,mha,RBF=init_model
# 3) instantiate everything
dim, basis = 2, 8 #scale to 3,16 at least # dim must be divisible by 2
depth=2 #scale to 2, at least
hidden_dim=3
num_heads=dim + basis 
num_edge_tokens=256
num_global_tokens=256
dropout=0.02
cutoff=10.0
num_neighbors=2


runid=datetime.datetime.now().strftime("%Y%m%d_%H%M%S")


from sklearn.neighbors import NearestNeighbors
from torch.utils.data import Dataset, DataLoader
import numpy as np, torch, glob

class InMemoryHoodDataset(Dataset):
    """
    Loads *.npz files, constructs fixed-size neighbourhoods around each
    site (anchor) and stores the result entirely in RAM.

    For a protein with S sites the shapes are
        z   : (S, N_NEIGHBORS)      int32
        pos : (S, N_NEIGHBORS, 3)   float32
        y   : (S,)                  float32
    """
    def __init__(self, paths, n_neighbors=N_NEIGHBORS, pin_memory=PIN_MEMORY):
        super().__init__()
        self.data = []
        nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm="brute")

        for p in paths:
            try:
                dat = np.load(p, allow_pickle=True)
                z_all   = dat["z"].astype(np.int32)        # (N,)
                pos_all = dat["pos"].astype(np.float32)    # (N,3)
                sites   = dat["sites"].astype(np.float32)  # (S,3)
                y       = dat["pks"].astype(np.float32)    # (S,)

                if len(sites) == 0:
                    continue  # skip empty entries

                nbrs.fit(pos_all)
                idx = nbrs.kneighbors(sites, return_distance=False)   # (S, N_NEIGHBORS)

                z_hood   = torch.from_numpy(z_all[idx])            # (S,N_NEIGHBORS)
                pos_hood = torch.from_numpy(pos_all[idx])          # (S,N_NEIGHBORS,3)
                y        = torch.from_numpy(y)                     # (S,)

                if pin_memory:
                    z_hood   = z_hood.pin_memory()
                    pos_hood = pos_hood.pin_memory()
                    y        = y.pin_memory()

                self.data.append((z_hood, pos_hood, y))
            except Exception as e:
                print(f"skipping {p}: {e}")

    def __len__(self):             return len(self.data)
    def __getitem__(self, idx):    return self.data[idx]

# ---------------------------------------------------------------------
# 2) collate function  -------------------------------------------------
# ---------------------------------------------------------------------
def pad_collate(batch):
    """
    Pads the variable-length site dimension so the batch can be stacked
    into one tensor.  A boolean mask keeps track of which elements are
    real data (True) vs. padding (False).
    """
    # batch = list[(z,pos,y), ...]         len = B
    B               = len(batch)
    S_max           = max(item[0].shape[0] for item in batch)   # longest protein
    device          = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    zs   = torch.zeros (B, S_max, N_NEIGHBORS ,   dtype=torch.int32 , device=device)
    pos  = torch.zeros (B, S_max, N_NEIGHBORS ,3, dtype=torch.float32, device=device)
    ys   = torch.full  ((B, S_max),  float("nan"), dtype=torch.float32, device=device)
    #ys   = torch.full  (B, S_max,               float("nan"),        dtype=torch.float32, device=device)
    mask = torch.zeros (B, S_max,                                   dtype=torch.bool,     device=device)

    for b,(z,pos_b,y) in enumerate(batch):
        S = z.shape[0]
        zs  [b, :S] = z.to(device)
        pos [b, :S] = pos_b.to(device)
        ys  [b, :S] = y.to(device)
        mask[b, :S] = True

    return zs, pos, ys, mask             # shapes – see above

# ---------------------------------------------------------------------
# 3) data loaders ------------------------------------------------------
# ---------------------------------------------------------------------

# 0) parameters you might want to expose at the top of the script
# ---------------------------------------------------------------------

# 3) data loaders ------------------------------------------------------
# ---------------------------------------------------------------------
np.random.seed(0)
all_paths = glob.glob("../inputs/*.npz")
np.random.shuffle(all_paths)
train_paths, val_paths = all_paths[:20], all_paths[20:30]

train_ds = InMemoryHoodDataset(train_paths)
val_ds   = InMemoryHoodDataset(val_paths)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE,
                          shuffle=True,  collate_fn=pad_collate,
                          num_workers=0, pin_memory=PIN_MEMORY)
val_loader   = DataLoader(val_ds  , batch_size=BATCH_SIZE,
                          shuffle=False, collate_fn=pad_collate,
                          num_workers=0, pin_memory=PIN_MEMORY)



# ---------------------------------------------------------------------
# 4) model pieces ------------------------------------------------------
# ---------------------------------------------------------------------
egnn_net = StackedEGNN(dim=dim, depth=depth, hidden_dim=hidden_dim,
                       dropout=dropout, num_positions=1000, num_tokens=118,
                       num_nearest_neighbors=num_neighbors,
                       norm_coors=True,
                       num_edge_tokens=num_edge_tokens,
                       num_global_tokens=num_global_tokens).to(device)

rbf_layer = LearnableRBF(num_basis=basis, cutoff=cutoff).to(device)
mha_layer = AttentionBlock(embed_dim=dim + basis,
                           num_heads=num_heads,
                           hidden_dim=hidden_dim).to(device)
pred_head = nn.Linear(dim + basis, 1).to(device)

protein_egnn=EGNN(dim=1,update_coors=False,num_nearest_neighbors=8)
criterion = nn.L1Loss()
optimizer = torch.optim.AdamW(
    list(egnn_net.parameters()) +
    list(rbf_layer.parameters()) +
    list(mha_layer.parameters()) +
    list(pred_head.parameters()) +
    list(protein_egnn.parameters()),
    lr=5e-3
)

epochs = 21  # or whatever you like

# ---------------------------------------------------------------------
# 5) forward for a *compressed* batch (R residues, N neighbours)
# ---------------------------------------------------------------------
# ---------------------------------------------------------------------
# 5) single-path forward – no shape guessing, no branching
# ---------------------------------------------------------------------
def forward_residues(z_r, x_r):
    """
    z_r : (R, N)       int32   – atomic numbers for R residues
    x_r : (R, N, 3)    float32 – coordinates
    returns (R, dim + basis)   – per-residue embeddings
    """
    # ---------- EGNN ----------
    h_out, coords = egnn_net(z_r, x_r)          # h_out is [tensor] or tensor
    h = h_out[0] if isinstance(h_out, (list, tuple)) else h_out   # (R, N, dim)

    # ---------- RBF on *input* coords (already (R,N,3)) ----------
    #d   = torch.cdist(x_r, x_r)            # (R, N, N)
    rbf = rbf_layer(coords)                     # (R, N, N, basis)

    # ---------- concat & attention ----------
    h0  = h.transpose(1, 2)                # (R, dim,   N)
    r0  = rbf[:, 0].transpose(1, 2)        # (R, basis, N)
    tok = torch.cat((r0, h0), dim=1)       # (R, dim+basis, N)

    tok, _ = mha_layer(tok.permute(2, 0, 1))   # (N, R, C) → attn(+PE)
    tok    = tok.permute(1, 0, 2).max(dim=1).values   # (R, C) max over neighbours
    return tok,     coords.mean(dim=1).unsqueeze(0)                               # (R, dim + basis)
                                         # (R, dim+basis)

# ---------------------------------------------------------------------
# 6) training / validation loop ---------------------------------------
# ---------------------------------------------------------------------
for epoch in range(epochs):
    # ======== TRAIN ========
    egnn_net.train(); rbf_layer.train(); mha_layer.train(); pred_head.train()
    tr_losses = []

    for z, x, y, mask in train_loader:                 # z:(B,S,N)  mask:(B,S)
        # compress away padding →  (R, N), (R, N, 3), (R,)
        valid      = mask.view(-1)
        z_res      = z.view(-1, z.size(2))[valid].to(device)
        x_res      = x.view(-1, x.size(2), 3)[valid].to(device)
        y_res      = y.view(-1)[valid].to(device)

        optimizer.zero_grad()

        #model
        feats, centroids = forward_residues(z_res, x_res)         # (R, C)
        
        preds = pred_head(feats)       
        t=preds.unsqueeze(0)
        #preds=protein_egnn(t,centroids)[0]

        loss  = criterion(preds.flatten(), y_res)

        if use_amp:
            scaler.scale(loss).backward()
            scaler.step(optimizer); scaler.update()
        else:
            loss.backward(); optimizer.step()

        tr_losses.append(loss.item())

    print(f"Epoch {epoch:3d} | train L1 = {np.mean(tr_losses):.4f}")

    
    # ======== VALID ========
    egnn_net.eval(); rbf_layer.eval(); mha_layer.eval(); pred_head.eval()
    vl_losses = []

    with torch.no_grad():
        for z, x, y, mask in val_loader:
            valid   = mask.view(-1)
            z_res   = z.view(-1, z.size(2))[valid].to(device)
            x_res   = x.view(-1, x.size(2), 3)[valid].to(device)
            y_res   = y.view(-1)[valid].to(device)

                #model
            feats, centroids = forward_residues(z_res, x_res)         # (R, C)
            
            preds = pred_head(feats)       
            t=preds.unsqueeze(0)
            #preds=protein_egnn(t,centroids)[0]

            loss  = criterion(preds.flatten(), y_res)
            vl_losses.append(loss.item())

    print(f"              |  val L1 = {np.mean(vl_losses):.4f}")


Epoch   0 | train L1 = 1.2519
              |  val L1 = 1.1477
Epoch   1 | train L1 = 1.1990
              |  val L1 = 1.1280
Epoch   2 | train L1 = 1.1771
              |  val L1 = 1.0822
Epoch   3 | train L1 = 1.1462
              |  val L1 = 1.0380
Epoch   4 | train L1 = 1.1211
              |  val L1 = 1.0375
Epoch   5 | train L1 = 1.1137
              |  val L1 = 0.9962
Epoch   6 | train L1 = 1.1062
              |  val L1 = 0.9722
Epoch   7 | train L1 = 1.0921
              |  val L1 = 0.9721
Epoch   8 | train L1 = 1.1130
              |  val L1 = 1.0192
Epoch   9 | train L1 = 1.1127
              |  val L1 = 1.0457
Epoch  10 | train L1 = 1.1440
              |  val L1 = 1.0814
Epoch  11 | train L1 = 1.1231
              |  val L1 = 1.0645
Epoch  12 | train L1 = 1.1170
              |  val L1 = 1.0472
Epoch  13 | train L1 = 1.1218
              |  val L1 = 0.9962
Epoch  14 | train L1 = 1.0872
              |  val L1 = 0.9940
Epoch  15 | train L1 = 1.0818
              |  val L1 =

In [125]:
lin2=nn.Linear(1,1)
lin2(preds)[99].shape

torch.Size([15, 1])

In [153]:
#dim = num neighbors
protein_egnn=EGNN(dim=15,update_coors=False,num_nearest_neighbors=8)
protein_egnn(preds.permute(2,0,1),centroids)[0].shape

torch.Size([1, 100, 15])

without mha (didnt adjust optimizer)

In [156]:
import datetime, time
from architecture import *
import torch
import glob, math, time, datetime
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from egnn_pytorch import EGNN_Network
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os
# 0) start timer
t0 = time.time()
N_NEIGHBORS = 15
BATCH_SIZE  =  1           # now safe to increase
PIN_MEMORY  = torch.cuda.is_available()
# reproducibility + device
torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# decide AMP only on GP0
use_amp = (device.type == "cuda")
if use_amp:
    scaler = GradScaler()
else:
    class DummyCM:
        def __enter__(self): pass
        def __exit__(self, *args): pass
    autocast = DummyCM
    scaler   = None

def init_model(dim,depth,hidden_dim,num_neighbors,num_edge_tokens,num_global_tokens,dropout):
    
    def build_egnn(dim,depth,hidden_dim,num_neighbors, num_edge_tokens,num_global_tokens,dropout):
        return StackedEGNN(
            dim=dim, depth=depth, hidden_dim=hidden_dim,
            dropout=dropout,
            num_positions=1000, num_tokens=118,
            num_nearest_neighbors=num_neighbors,
            norm_coors=True,
            num_edge_tokens=num_edge_tokens,
            num_global_tokens=num_global_tokens
        )
    net   = build_egnn(dim,depth,hidden_dim,num_neighbors,num_edge_tokens,num_global_tokens,dropout).to(device)
    mha   = AttentionBlock(embed_dim=dim+basis, num_heads=num_heads, hidden_dim=hidden_dim).to(device)
    RBF   = LearnableRBF(num_basis=basis, cutoff=cutoff).to(device) 
    return net, mha, RBF
#net,mha,RBF=init_model
# 3) instantiate everything
dim, basis = 2, 8 #scale to 3,16 at least # dim must be divisible by 2
depth=2 #scale to 2, at least
hidden_dim=3
num_heads=dim + basis 
num_edge_tokens=256
num_global_tokens=256
dropout=0.02
cutoff=10.0
num_neighbors=2


runid=datetime.datetime.now().strftime("%Y%m%d_%H%M%S")


from sklearn.neighbors import NearestNeighbors
from torch.utils.data import Dataset, DataLoader
import numpy as np, torch, glob

class InMemoryHoodDataset(Dataset):
    """
    Loads *.npz files, constructs fixed-size neighbourhoods around each
    site (anchor) and stores the result entirely in RAM.

    For a protein with S sites the shapes are
        z   : (S, N_NEIGHBORS)      int32
        pos : (S, N_NEIGHBORS, 3)   float32
        y   : (S,)                  float32
    """
    def __init__(self, paths, n_neighbors=N_NEIGHBORS, pin_memory=PIN_MEMORY):
        super().__init__()
        self.data = []
        nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm="brute")

        for p in paths:
            try:
                dat = np.load(p, allow_pickle=True)
                z_all   = dat["z"].astype(np.int32)        # (N,)
                pos_all = dat["pos"].astype(np.float32)    # (N,3)
                sites   = dat["sites"].astype(np.float32)  # (S,3)
                y       = dat["pks"].astype(np.float32)    # (S,)

                if len(sites) == 0:
                    continue  # skip empty entries

                nbrs.fit(pos_all)
                idx = nbrs.kneighbors(sites, return_distance=False)   # (S, N_NEIGHBORS)

                z_hood   = torch.from_numpy(z_all[idx])            # (S,N_NEIGHBORS)
                pos_hood = torch.from_numpy(pos_all[idx])          # (S,N_NEIGHBORS,3)
                y        = torch.from_numpy(y)                     # (S,)

                if pin_memory:
                    z_hood   = z_hood.pin_memory()
                    pos_hood = pos_hood.pin_memory()
                    y        = y.pin_memory()

                self.data.append((z_hood, pos_hood, y))
            except Exception as e:
                print(f"skipping {p}: {e}")

    def __len__(self):             return len(self.data)
    def __getitem__(self, idx):    return self.data[idx]

# ---------------------------------------------------------------------
# 2) collate function  -------------------------------------------------
# ---------------------------------------------------------------------
def pad_collate(batch):
    """
    Pads the variable-length site dimension so the batch can be stacked
    into one tensor.  A boolean mask keeps track of which elements are
    real data (True) vs. padding (False).
    """
    # batch = list[(z,pos,y), ...]         len = B
    B               = len(batch)
    S_max           = max(item[0].shape[0] for item in batch)   # longest protein
    device          = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    zs   = torch.zeros (B, S_max, N_NEIGHBORS ,   dtype=torch.int32 , device=device)
    pos  = torch.zeros (B, S_max, N_NEIGHBORS ,3, dtype=torch.float32, device=device)
    ys   = torch.full  ((B, S_max),  float("nan"), dtype=torch.float32, device=device)
    #ys   = torch.full  (B, S_max,               float("nan"),        dtype=torch.float32, device=device)
    mask = torch.zeros (B, S_max,                                   dtype=torch.bool,     device=device)

    for b,(z,pos_b,y) in enumerate(batch):
        S = z.shape[0]
        zs  [b, :S] = z.to(device)
        pos [b, :S] = pos_b.to(device)
        ys  [b, :S] = y.to(device)
        mask[b, :S] = True

    return zs, pos, ys, mask             # shapes – see above

# ---------------------------------------------------------------------
# 3) data loaders ------------------------------------------------------
# ---------------------------------------------------------------------

# 0) parameters you might want to expose at the top of the script
# ---------------------------------------------------------------------

# 3) data loaders ------------------------------------------------------
# ---------------------------------------------------------------------
np.random.seed(0)
all_paths = glob.glob("../inputs/*.npz")
np.random.shuffle(all_paths)
train_paths, val_paths = all_paths[:20], all_paths[20:30]

train_ds = InMemoryHoodDataset(train_paths)
val_ds   = InMemoryHoodDataset(val_paths)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE,
                          shuffle=True,  collate_fn=pad_collate,
                          num_workers=0, pin_memory=PIN_MEMORY)
val_loader   = DataLoader(val_ds  , batch_size=BATCH_SIZE,
                          shuffle=False, collate_fn=pad_collate,
                          num_workers=0, pin_memory=PIN_MEMORY)



# ---------------------------------------------------------------------
# 4) model pieces ------------------------------------------------------
# ---------------------------------------------------------------------
egnn_net = StackedEGNN(dim=dim, depth=depth, hidden_dim=hidden_dim,
                       dropout=dropout, num_positions=1000, num_tokens=118,
                       num_nearest_neighbors=num_neighbors,
                       norm_coors=True,
                       num_edge_tokens=num_edge_tokens,
                       num_global_tokens=num_global_tokens).to(device)

rbf_layer = LearnableRBF(num_basis=basis, cutoff=cutoff).to(device)
mha_layer = AttentionBlock(embed_dim=dim + basis,
                           num_heads=num_heads,
                           hidden_dim=hidden_dim).to(device)
pred_head = nn.Linear(dim + basis, 1).to(device)

protein_egnn=EGNN(dim=1,update_coors=False,num_nearest_neighbors=8)
criterion = nn.L1Loss()
optimizer = torch.optim.AdamW(
    list(egnn_net.parameters()) +
    list(rbf_layer.parameters()) +
    list(mha_layer.parameters()) +
    list(pred_head.parameters()) +
    list(protein_egnn.parameters()),
    lr=5e-3
)

epochs = 21  # or whatever you like

# ---------------------------------------------------------------------
# 5) forward for a *compressed* batch (R residues, N neighbours)
# ---------------------------------------------------------------------
# ---------------------------------------------------------------------
# 5) single-path forward – no shape guessing, no branching
# ---------------------------------------------------------------------
def forward_residues(z_r, x_r):
    """
    z_r : (R, N)       int32   – atomic numbers for R residues
    x_r : (R, N, 3)    float32 – coordinates
    returns (R, dim + basis)   – per-residue embeddings
    """
    # ---------- EGNN ----------
    h_out, coords = egnn_net(z_r, x_r)          # h_out is [tensor] or tensor
    h = h_out[0] if isinstance(h_out, (list, tuple)) else h_out   # (R, N, dim)

    # ---------- RBF on *input* coords (already (R,N,3)) ----------
    #d   = torch.cdist(x_r, x_r)            # (R, N, N)
    rbf = rbf_layer(coords)                     # (R, N, N, basis)

    # ---------- concat & attention ----------
    h0  = h.transpose(1, 2)                # (R, dim,   N)
    r0  = rbf[:, 0].transpose(1, 2)        # (R, basis, N)
    tok = torch.cat((r0, h0), dim=1).permute(2,0,1)      # (R, dim+basis, N)

    #tok, _ = mha_layer(tok.permute(2, 0, 1))   # (N, R, C) → attn(+PE)
    tok    = tok.permute(1, 0, 2).max(dim=1).values   # (R, C) max over neighbours
    return tok,     coords.mean(dim=1).unsqueeze(0)                               # (R, dim + basis)
                                         # (R, dim+basis)

# ---------------------------------------------------------------------
# 6) training / validation loop ---------------------------------------
# ---------------------------------------------------------------------
for epoch in range(epochs):
    # ======== TRAIN ========
    egnn_net.train(); rbf_layer.train(); mha_layer.train(); pred_head.train()
    tr_losses = []

    for z, x, y, mask in train_loader:                 # z:(B,S,N)  mask:(B,S)
        # compress away padding →  (R, N), (R, N, 3), (R,)
        valid      = mask.view(-1)
        z_res      = z.view(-1, z.size(2))[valid].to(device)
        x_res      = x.view(-1, x.size(2), 3)[valid].to(device)
        y_res      = y.view(-1)[valid].to(device)

        optimizer.zero_grad()

        #model
        feats, centroids = forward_residues(z_res, x_res)         # (R, C)
        
        preds = pred_head(feats)       
        t=preds.unsqueeze(0)
        preds=protein_egnn(t,centroids)[0]

        loss  = criterion(preds.flatten(), y_res)

        if use_amp:
            scaler.scale(loss).backward()
            scaler.step(optimizer); scaler.update()
        else:
            loss.backward(); optimizer.step()

        tr_losses.append(loss.item())

    print(f"Epoch {epoch:3d} | train L1 = {np.mean(tr_losses):.4f}")

    
    # ======== VALID ========
    egnn_net.eval(); rbf_layer.eval(); mha_layer.eval(); pred_head.eval()
    vl_losses = []

    with torch.no_grad():
        for z, x, y, mask in val_loader:
            valid   = mask.view(-1)
            z_res   = z.view(-1, z.size(2))[valid].to(device)
            x_res   = x.view(-1, x.size(2), 3)[valid].to(device)
            y_res   = y.view(-1)[valid].to(device)

                #model
            feats, centroids = forward_residues(z_res, x_res)         # (R, C)
            
            preds = pred_head(feats)       
            t=preds.unsqueeze(0)
            preds=protein_egnn(t,centroids)[0]

            loss  = criterion(preds.flatten(), y_res)
            vl_losses.append(loss.item())

    print(f"              |  val L1 = {np.mean(vl_losses):.4f}")


Epoch   0 | train L1 = 1.2283
              |  val L1 = 1.1830
Epoch   1 | train L1 = 1.2138
              |  val L1 = 1.1819
Epoch   2 | train L1 = 1.2122
              |  val L1 = 1.1786
Epoch   3 | train L1 = 1.2105
              |  val L1 = 1.1794
Epoch   4 | train L1 = 1.2093
              |  val L1 = 1.1796
Epoch   5 | train L1 = 1.2098
              |  val L1 = 1.1783
Epoch   6 | train L1 = 1.2084
              |  val L1 = 1.1809
Epoch   7 | train L1 = 1.2054
              |  val L1 = 1.1781
Epoch   8 | train L1 = 1.2051
              |  val L1 = 1.1759
Epoch   9 | train L1 = 1.2057
              |  val L1 = 1.1783
Epoch  10 | train L1 = 1.2049
              |  val L1 = 1.1760
Epoch  11 | train L1 = 1.2064
              |  val L1 = 1.1768
Epoch  12 | train L1 = 1.2023
              |  val L1 = 1.1829
Epoch  13 | train L1 = 1.2012
              |  val L1 = 1.1749
Epoch  14 | train L1 = 1.2035
              |  val L1 = 1.1781
Epoch  15 | train L1 = 1.1996
              |  val L1 =